In [1]:
from glob import glob
from PIL import Image
from pathlib import Path
import pandas as pd
import json

At this point we already have captioned images in Chinese, and need to simply reformat for SD finetuning

Convert dataset into expected format

In [2]:
captioned_en = pd.read_json("captioned_albums.json")
captioned_cn = pd.read_json("captioned_albums_cn.json").T

In [3]:
captioned_out = pd.DataFrame()
captioned_out["file_name"] = captioned_en["file"].apply(lambda a: a.split('\\')[-1])
captioned_out["text"] = captioned_cn[0].apply(lambda a: a["translation_text"])

In [4]:
captioned_out

,file_name,text
0,0.jpg,黑裙子女性的绘画
1,1.jpg,一只狗鼻子的黑白相片
2,10.jpg,一张坐在沙发上的两个妇女画画
3,100.jpg,一部心心不宁的电影海报
4,1000.jpg,黑红海报 上面有恶魔的脸
...,...,...
3983,13582.jpg,一张坐在地上的人的图画
3984,13583.jpg,一名裸女在墙前摆着姿势
3985,13584.jpg,一幅男人戴帽子的画
3986,13585.jpg,一张坐在长椅上的妇女的黑白照片


In [5]:
captioned_out.to_json("metadata.jsonl", lines=True, orient="records")

## Create prompts


In [14]:
base = "音乐专辑"

In [15]:
SEED=0
SAMPLE_SIZE=200

In [16]:
p080 = pd.read_json("../BLOOM_results/3K_BLOOM_p080.jsonl", lines=True).sample(SAMPLE_SIZE, random_state=SEED)
p085 = pd.read_json("../BLOOM_results/3K_BLOOM_p085.jsonl", lines=True).sample(SAMPLE_SIZE, random_state=SEED)
p090 = pd.read_json("../BLOOM_results/3K_BLOOM_p090.jsonl", lines=True).sample(SAMPLE_SIZE, random_state=SEED)
p095 = pd.read_json("../BLOOM_results/3K_BLOOM_p095.jsonl", lines=True).sample(SAMPLE_SIZE, random_state=SEED)
p099 = pd.read_json("../BLOOM_results/3K_BLOOM_p099.jsonl", lines=True).sample(SAMPLE_SIZE, random_state=SEED)

In [17]:
sample = pd.concat([p080, p085, p090, p095, p099])

In [18]:
prompts = [base + " " + a for a in sample["response"].apply(lambda a: a[0]["generated_text"]).str.split("下面是一首歌的中文歌词。\n歌词：\n").apply(lambda a: a[1]).str.replace("\\n", " ", regex=True).to_list()]

In [19]:
with open('1000prompts.txt', 'w', encoding="utf-8") as f:
    for line in prompts:
        print(line,file=f)